#Predicción de Tiempos Verbales

En base a los resultados obtenidos en el notebook méticas modelo, se decide utilizar para la clasificación de Tiempo y Modo la estrategia de contextualización de embeddings Sum all, para Persona Second last y para Número Sum all. En este notebook se ve todo el proceso desde el armado de cada dataset, el entrenamiento de los modelos con las estrategias seleccionadas y la evaluación del modelo para ver como responde ante el ingreso de una oración, si predice correctamente Tiempo, Modo, Persona y Número de cada verbo dentro de dicha oración

###1.Utilizamos GPU

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


###2.Instalación de librerías y descarga del dataset

In [ ]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=da01939ed7f6375d0557768e2a0cd069257e1055ada268263a77d79517fbf658
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [ ]:
import wget
import os

print('Downloading dataset...')

# The URL for the dataset zip file.
url = 'https://github.com/AndreTati/PrediccionTiemposVerbales/archive/refs/heads/main.zip'

# Download the file (if we haven't already)
if not os.path.exists('./main.zip'):
    wget.download(url, './main.zip')

In [ ]:
if not os.path.exists('./main/'):
    !unzip main.zip

Archive:  main.zip
ecd47eedde475c1fba15582c9b510065c6f20077
   creating: PrediccionTiemposVerbales-main/
  inflating: PrediccionTiemposVerbales-main/README.md  
   creating: PrediccionTiemposVerbales-main/data/
  inflating: PrediccionTiemposVerbales-main/data/ancora_verbos.txt  
 extracting: PrediccionTiemposVerbales-main/data/master.zip  
   creating: PrediccionTiemposVerbales-main/dev/
  inflating: PrediccionTiemposVerbales-main/dev/model_dev.ipynb  
   creating: PrediccionTiemposVerbales-main/prod/
  inflating: PrediccionTiemposVerbales-main/prod/app.py  
  inflating: PrediccionTiemposVerbales-main/prod/label_dicts.pkl  
  inflating: PrediccionTiemposVerbales-main/prod/model.py  
  inflating: PrediccionTiemposVerbales-main/prod/modelNumber.pth  
  inflating: PrediccionTiemposVerbales-main/prod/modelPerson.pth  
  inflating: PrediccionTiemposVerbales-main/prod/modelTime.pth  
  inflating: PrediccionTiemposVerbales-main/prod/requirements.txt  
  inflating: PrediccionTiemposVerbales-ma

In [ ]:
import pandas as pd
# Leer el archivo .txt sin encabezado (sin nombres de columnas)
df = pd.read_csv("/content/PrediccionTiemposVerbales-main/data/ancora_verbos.txt", sep="\t", header=None)

# Asignar nombres de columnas manualmente
df.columns = ["sentence", "verb", "Tense", "Mood", "Person", "Number"]

train_data = df.to_dict(orient="records")

###3.Tokenizamos con BERT

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np

# Cargar tokenizer en español
tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
model = BertModel.from_pretrained("dccuchile/bert-base-spanish-wwm-cased", output_hidden_states=True)
model.to(device)
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(31002, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:
def get_bert_embeddings(sentence):
    inputs = tokenizer(sentence, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        hidden_states = outputs.hidden_states
    return inputs, hidden_states

def get_verb_embedding(inputs, hidden_states, verb, strategy):
    # Convertir input_ids a tokens
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    #print(tokens)
    # Tokenizar el verbo para saber cómo está dividido
    verb_subtokens = tokenizer.tokenize(verb)
    #print(verb_subtokens)
    # Encontrar los índices de los subtokens del verbo en la secuencia
    verb_indices=[]
    # Buscar los índices de los subtokens del verbo en la secuencia
    for i in range(len(tokens) - len(verb_subtokens) + 1):
        if tokens[i:i + len(verb_subtokens)] == verb_subtokens:
            verb_indices = list(range(i, i + len(verb_subtokens)))
            #print(verb_indices)
            break
    if not verb_indices:
        return None  # Verbo no encontrado

    # === Embedding según estrategia ===
    if strategy == "second_last":
        emb = torch.stack([hidden_states[-2][0][i] for i in verb_indices], dim=0)
        return emb.mean(dim=0).cpu()

    elif strategy == "sum_last4":
        emb = sum(hidden_states[-i][0][verb_indices].mean(dim=0) for i in range(1, 5))
        return emb.cpu()

    elif strategy == "concat_last4":
        emb = [hidden_states[-i][0][verb_indices].mean(dim=0) for i in range(1, 5)]
        return torch.cat(emb, dim=-1).cpu()

    elif strategy == "sum_all":
        emb = sum(hidden_states[i][0][verb_indices].mean(dim=0) for i in range(1, 13))
        return emb.cpu()

    else:
        raise ValueError(f"Estrategia desconocida: {strategy}")

In [ ]:
dataset_time_mood, dataset_mood, dataset_person, dataset_number = [], [], [], []

for row in train_data:
    inputs, hidden_states = get_bert_embeddings(row["sentence"])
    if hidden_states is not None:
        combined_label = row["Tense"] + "_" + row["Mood"]
        embTime_mood = get_verb_embedding(inputs, hidden_states, row["verb"], "sum_all")
        dataset_time_mood.append([row["verb"], embTime_mood, combined_label])
        #embMood = get_verb_embedding(inputs, hidden_states, row["verb"], "sum_last4")
        #dataset_mood.append([row["verb"], embMood, row["Mood"]])
        embPerson = get_verb_embedding(inputs, hidden_states, row["verb"], "second_last")
        dataset_person.append([row["verb"], embPerson, row["Person"]])
        embNumber = get_verb_embedding(inputs, hidden_states, row["verb"], "sum_all")
        dataset_number.append([row["verb"], embNumber, row["Number"]])

In [ ]:
print(dataset_time_mood[0])
#print(dataset_mood[0])
print(dataset_person[0])
print(dataset_number[0])

['subieron', tensor([-1.0047e+00,  6.8851e+00, -9.6178e+00,  5.4297e+00, -1.3043e+00,
        -8.4807e-01, -3.9509e+00, -8.3841e+00,  1.9758e+00, -3.9723e+00,
         2.6727e+00,  2.2721e+00,  1.1563e+01, -3.2806e+00,  9.7570e+00,
         1.3594e+01,  8.3843e+00, -4.2931e+00,  7.8112e-01,  5.0232e+00,
         1.4612e+01,  2.5574e+00,  2.4415e+00,  2.8016e+00, -9.5765e+00,
         5.6787e+00,  4.5310e+00,  9.0642e+00, -1.9742e+00, -1.4959e+00,
         1.0739e+01, -2.7982e+00, -2.9520e+00,  1.2341e+01, -6.3644e+00,
        -3.5748e+00,  7.2429e-01,  6.9207e+00,  5.3451e+00, -7.1934e+00,
         7.1091e+00, -3.2557e+00,  1.9711e-01,  7.5680e+00,  5.9736e+00,
        -1.1617e+01,  3.7201e+00,  6.8119e+00,  8.4517e+00, -3.3732e+00,
         1.2469e+00,  5.3437e+00, -6.3511e+00, -1.0753e+01, -2.4570e+00,
        -1.9807e+00, -3.2167e+00,  5.5849e+00, -8.4424e+00,  1.1563e+00,
        -4.9422e+00,  6.8992e+00, -4.0021e+00, -5.7115e+00,  1.1186e+01,
        -5.7990e+00, -1.2140e+01,  6.0

In [ ]:
# Extraer etiquetas únicas de cada lista
all_time_mood = sorted(set(row[2] for row in dataset_time_mood))
#all_moods = sorted(set(row[2] for row in dataset_mood))
all_persons = sorted(set(row[2] for row in dataset_person))
all_numbers = sorted(set(row[2] for row in dataset_number))

# Crear diccionarios {etiqueta_str: clase_int}
time_mood2id = {label: idx for idx, label in enumerate(all_time_mood)}
#mood2id = {label: idx for idx, label in enumerate(all_moods)}
person2id = {label: idx for idx, label in enumerate(all_persons)}
number2id = {label: idx for idx, label in enumerate(all_numbers)}

dataset_time_mood = [[verb, emb, time_mood2id[label]] for verb, emb, label in dataset_time_mood]
#dataset_mood = [[verb, emb, mood2id[label]] for verb, emb, label in dataset_mood]
dataset_person = [[verb, emb, person2id[label]] for verb, emb, label in dataset_person]
dataset_number = [[verb, emb, number2id[label]] for verb, emb, label in dataset_number]

In [ ]:
print(all_time_mood)
print(time_mood2id)
#print(all_moods)
#print(mood2id)
print(all_persons)
print(person2id)
print(all_numbers)
print(number2id)
#print(dataset_time[0])

['Fut_Ind', 'Imp_Ind', 'Imp_Sub', 'Past_Ind', 'Pres_Ind', 'Pres_Sub']
{'Fut_Ind': 0, 'Imp_Ind': 1, 'Imp_Sub': 2, 'Past_Ind': 3, 'Pres_Ind': 4, 'Pres_Sub': 5}
[1, 2, 3]
{1: 0, 2: 1, 3: 2}
['Plur', 'Sing']
{'Plur': 0, 'Sing': 1}


In [ ]:
from torch.utils.data import Dataset

class SimpleVerbDataset(Dataset):
    def __init__(self, data):
        self.data = data  # Lista de tuplas (verbo, embedding, etiqueta)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        _, embedding, label = self.data[idx]

        # Asegurarse de que embedding ya esté en tipo float y label como long
        if not isinstance(embedding, torch.Tensor):
            embedding = torch.tensor(embedding, dtype=torch.float32)
        else:
            embedding = embedding.float()

        if not isinstance(label, torch.Tensor):
            label = torch.tensor(label, dtype=torch.long)
        else:
            label = label.long()

        return embedding, label

In [ ]:
id2time_mood = {v: k for k, v in time_mood2id.items()}
#id2mood = {v: k for k, v in mood2id.items()}
id2person = {v: k for k, v in person2id.items()}
id2number = {v: k for k, v in number2id.items()}

In [ ]:
print(id2time_mood)

{0: 'Fut_Ind', 1: 'Imp_Ind', 2: 'Imp_Sub', 3: 'Past_Ind', 4: 'Pres_Ind', 5: 'Pres_Sub'}


###4. Obtenemos los dataloaders de entrenamiento

In [ ]:
time_mood_dataset = SimpleVerbDataset(dataset_time_mood)
person_dataset = SimpleVerbDataset(dataset_person)
number_dataset = SimpleVerbDataset(dataset_number)

from torch.utils.data import DataLoader
batch_size=32

time_mood_loader = DataLoader(time_mood_dataset, batch_size=batch_size, shuffle=True)
person_loader = DataLoader(person_dataset, batch_size=batch_size, shuffle=True)
number_loader = DataLoader(number_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
print(time_mood_dataset[0])
#print(mood_dataset[0])
print(person_dataset[0])
print(number_dataset[0])

(tensor([-1.0047e+00,  6.8851e+00, -9.6178e+00,  5.4297e+00, -1.3043e+00,
        -8.4807e-01, -3.9509e+00, -8.3841e+00,  1.9758e+00, -3.9723e+00,
         2.6727e+00,  2.2721e+00,  1.1563e+01, -3.2806e+00,  9.7570e+00,
         1.3594e+01,  8.3843e+00, -4.2931e+00,  7.8112e-01,  5.0232e+00,
         1.4612e+01,  2.5574e+00,  2.4415e+00,  2.8016e+00, -9.5765e+00,
         5.6787e+00,  4.5310e+00,  9.0642e+00, -1.9742e+00, -1.4959e+00,
         1.0739e+01, -2.7982e+00, -2.9520e+00,  1.2341e+01, -6.3644e+00,
        -3.5748e+00,  7.2429e-01,  6.9207e+00,  5.3451e+00, -7.1934e+00,
         7.1091e+00, -3.2557e+00,  1.9711e-01,  7.5680e+00,  5.9736e+00,
        -1.1617e+01,  3.7201e+00,  6.8119e+00,  8.4517e+00, -3.3732e+00,
         1.2469e+00,  5.3437e+00, -6.3511e+00, -1.0753e+01, -2.4570e+00,
        -1.9807e+00, -3.2167e+00,  5.5849e+00, -8.4424e+00,  1.1563e+00,
        -4.9422e+00,  6.8992e+00, -4.0021e+00, -5.7115e+00,  1.1186e+01,
        -5.7990e+00, -1.2140e+01,  6.0295e+00,  2.

###5. Planteo del modelo y entrenamiento

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class VerbClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)

In [ ]:
embedding_dim = 768
embedding_dim_concat_last4 = 3072

modelTime_mood = VerbClassifier(input_dim=embedding_dim, num_classes=len(time_mood2id)).to(device)
#modelMood = VerbClassifier(input_dim=embedding_dim, num_classes=len(mood2id)).to(device)
modelPerson = VerbClassifier(input_dim=embedding_dim, num_classes=len(person2id)).to(device)
modelNumber = VerbClassifier(input_dim=embedding_dim, num_classes=len(number2id)).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer_time = torch.optim.Adam(modelTime_mood.parameters(), lr=1e-4, weight_decay=1e-5)
#optimizer_mood = torch.optim.Adam(modelMood.parameters(), lr=1e-4, weight_decay=1e-5)
optimizer_person = torch.optim.Adam(modelPerson.parameters(), lr=1e-4, weight_decay=1e-5)
optimizer_number = torch.optim.Adam(modelNumber.parameters(), lr=1e-4, weight_decay=1e-5)

In [ ]:
def train_and_evaluate(dataloader, label_name, model, loss_fn, optimizer, epochs=20):

    # Entrenamiento
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch_x, batch_y in dataloader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = loss_fn(outputs, batch_y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1} | Loss: {total_loss:.4f}")

    # Evaluación
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch_x, batch_y in dataloader:
            outputs = model(batch_x.to(device))
            preds = outputs.argmax(dim=1).cpu().numpy()
            labels = batch_y.cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels)


In [ ]:
epochs=10
train_and_evaluate(time_mood_loader, "Tiempo", modelTime_mood, loss_fn, optimizer_time, epochs)
#train_and_evaluate(mood_loader, "Modo", modelMood, loss_fn, optimizer_mood, epochs)
train_and_evaluate(person_loader, "Persona", modelPerson, loss_fn, optimizer_person, 5)
train_and_evaluate(number_loader, "Número", modelNumber, loss_fn, optimizer_number, 5)

Epoch 1 | Loss: 77.8029
Epoch 2 | Loss: 11.0035
Epoch 3 | Loss: 6.2514
Epoch 4 | Loss: 3.7240
Epoch 5 | Loss: 1.8594
Epoch 6 | Loss: 1.6077
Epoch 7 | Loss: 1.0880
Epoch 8 | Loss: 1.3838
Epoch 9 | Loss: 0.5476
Epoch 10 | Loss: 0.4666
Epoch 1 | Loss: 81.6384
Epoch 2 | Loss: 20.8233
Epoch 3 | Loss: 15.9306
Epoch 4 | Loss: 13.1665
Epoch 5 | Loss: 11.0611
Epoch 1 | Loss: 24.7066
Epoch 2 | Loss: 1.2662
Epoch 3 | Loss: 0.4489
Epoch 4 | Loss: 0.2179
Epoch 5 | Loss: 0.8506


In [ ]:
torch.save(modelTime_mood.state_dict(), "modelTime.pth")
#torch.save(modelMood.state_dict(), "modelMood.pth")
torch.save(modelPerson.state_dict(), "modelPerson.pth")
torch.save(modelNumber.state_dict(), "modelNumber.pth")

In [ ]:
import pickle

with open("label_dicts.pkl", "wb") as f:
    pickle.dump({
        "tense": id2time_mood,
        "person": id2person,
        "number": id2number
    }, f)

###6. Evaluación

Para probar la red se descarga Spacy que se utilizará para detectar los verbos dentro de la oración

In [ ]:
!pip install -U spacy
!python -m spacy download es_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 2.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
nlp = spacy.load("es_core_news_lg")

In [ ]:
def detectar_verbos_spacy(oracion):
    doc = nlp(oracion)
    verbos = []

    for i, token in enumerate(doc):
        if token.pos_ == "VERB":
            if i > 0 and doc[i - 1].pos_ == "AUX":
                aux_token = doc[i - 1]
                verbos.append((aux_token.text, aux_token.i))
            else:
                verbos.append((token.text, token.i))

    # Eliminar duplicados si hay
    verbos = list(dict.fromkeys(verbos))
    return verbos

Se definen 4 diccionarios para que los resultados de la red sean más entendibles al usuarios

In [ ]:
# Diccionario para completar abreviaciones
descripcion_tiempos = {
    "Pres": "Presente",
    "Past": "Pasado",
    "Imp": "Imperfecto",
    "Fut": "Futuro"
}

descripcion_modos = {
    "Ind": "Indicativo",
    "Sub": "Subjuntivo"
}

descripcion_personas = {
    1: "Primera",
    2: "Segunda",
    3: "Tercera"
}

descripcion_numeros = {
    "Sing": "Singular",
    "Plur": "Plural"
}

Finalmente se define una función que recibe la oración. Dicha oración se pasa por spacy para detectar los verbos y luego por cada verbo se generan los embeddings correspondientes y se pasa por cada modelo entrenado. Se devuelven los resultados obtenidos

In [ ]:
def analizar_oracion(oracion):
    verbos = detectar_verbos_spacy(oracion)
    resultados = []

    # Solo pasamos por BERT una vez por oración
    inputs, hidden_states = get_bert_embeddings(oracion)

    for verbo, _ in verbos:
        # Extraer embeddings usando cada estrategia correspondiente
        emb_time = get_verb_embedding(inputs, hidden_states, verbo, "sum_all")
        emb_person = get_verb_embedding(inputs, hidden_states, verbo, "second_last")
        emb_number = get_verb_embedding(inputs, hidden_states, verbo, "sum_all")

        # Si alguno no se encuentra (por problemas con subwords), salteamos
        if None in [emb_time, emb_person, emb_number]:
            continue
        logits=modelTime_mood(emb_time.unsqueeze(0).to(device))
        print(torch.softmax(logits, dim=1))

        tiempo = id2time_mood[modelTime_mood(emb_time.unsqueeze(0).to(device)).argmax(dim=1).item()]
        persona = id2person[modelPerson(emb_person.unsqueeze(0).to(device)).argmax(dim=1).item()]
        numero = id2number[modelNumber(emb_number.unsqueeze(0).to(device)).argmax(dim=1).item()]
        tiempo, modo = tiempo.split("_")

        resultados.append({
            "verbo": verbo,
            "tiempo": tiempo,
            "modo": modo,
            "persona": persona,
            "numero": numero
        })

    return resultados

Finalmente se muestran los resultados utilizando los diccionarios definidos anteriormente

In [ ]:
oracion = "Cuando llegamos a casa, ella estaba cocinando y yo lavaba los platos."
for resultado in analizar_oracion(oracion):
    print(f"🔹 Verbo: {resultado['verbo']}")
    print(f"  • Tiempo: {resultado['tiempo']} ({descripcion_tiempos.get(resultado['tiempo'], 'Desconocido')})")
    print(f"  • Modo: {resultado['modo']} ({descripcion_modos.get(resultado['modo'], 'Desconocido')})")
    print(f"  • Persona: {resultado['persona']} ({descripcion_personas.get(resultado['persona'], 'Desconocido')})")
    print(f"  • Número: {resultado['numero']} ({descripcion_numeros.get(resultado['numero'], 'Desconocido')})")

tensor([[3.8895e-04, 2.4617e-03, 3.0651e-05, 4.3261e-01, 5.6418e-01, 3.2210e-04]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)
tensor([[1.2697e-12, 1.0000e+00, 3.5142e-09, 2.1621e-09, 5.5062e-11, 3.4771e-12]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)
tensor([[5.7029e-10, 1.0000e+00, 1.8360e-07, 5.8181e-09, 4.3657e-09, 1.3618e-12]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)
🔹 Verbo: llegamos
  • Tiempo: Pres (Presente)
  • Modo: Ind (Indicativo)
  • Persona: 1 (Primera)
  • Número: Plur (Plural)
🔹 Verbo: estaba
  • Tiempo: Imp (Imperfecto)
  • Modo: Ind (Indicativo)
  • Persona: 3 (Tercera)
  • Número: Sing (Singular)
🔹 Verbo: lavaba
  • Tiempo: Imp (Imperfecto)
  • Modo: Ind (Indicativo)
  • Persona: 1 (Primera)
  • Número: Sing (Singular)


La misma función definida anterior se modifica para validar como hace el análisis sintáctico Spacy

In [ ]:
def detectar_verbos_spacy2(oracion):
    doc = nlp(oracion)
    return doc

In [ ]:
oracion = "Cuando llegue a casa, ella cocinará y yo lavaré los platos."
doc=detectar_verbos_spacy2(oracion)
for token in doc:
  print(token.text, token.i, token.pos_)

Cuando 0 SCONJ
llegue 1 VERB
a 2 ADP
casa 3 NOUN
, 4 PUNCT
ella 5 PRON
cocinará 6 VERB
y 7 CCONJ
yo 8 PRON
lavaré 9 VERB
los 10 DET
platos 11 NOUN
. 12 PUNCT


In [ ]:
oracion = "Cuando llegamos a casa, ella estaba cocinando y yo lavaba los platos."
doc=detectar_verbos_spacy2(oracion)
for token in doc:
  print(token.text, token.i, token.pos_)

Cuando 0 SCONJ
llegamos 1 VERB
a 2 ADP
casa 3 NOUN
, 4 PUNCT
ella 5 PRON
estaba 6 AUX
cocinando 7 VERB
y 8 CCONJ
yo 9 PRON
lavaba 10 VERB
los 11 DET
platos 12 NOUN
. 13 PUNCT


In [ ]:
oracion = "Cuando llegue a casa, ella cocinará y yo lavaré los platos."
for resultado in analizar_oracion(oracion):
    print(f"🔹 Verbo: {resultado['verbo']}")
    print(f"  • Tiempo: {resultado['tiempo']}")
    print(f"  • Modo: {resultado['modo']}")
    print(f"  • Persona: {resultado['persona']}")
    print(f"  • Número: {resultado['numero']}")

tensor([[1.0317e-05, 9.4379e-08, 7.8495e-07, 1.0852e-08, 2.8391e-06, 9.9999e-01]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)
tensor([[1.0000e+00, 4.1239e-09, 5.8817e-09, 8.3394e-08, 1.1168e-08, 2.3174e-10]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)
tensor([[1.0000e+00, 3.1227e-08, 8.6159e-08, 3.2609e-07, 3.2767e-08, 9.4441e-08]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)
🔹 Verbo: llegue
  • Tiempo: Pres
  • Modo: Sub
  • Persona: 1
  • Número: Sing
🔹 Verbo: cocinará
  • Tiempo: Fut
  • Modo: Ind
  • Persona: 3
  • Número: Sing
🔹 Verbo: lavaré
  • Tiempo: Fut
  • Modo: Ind
  • Persona: 1
  • Número: Sing
